## Analyzing Many Models



In [ ]:
import pandas as pd
import numpy as np
from google.cloud import bigquery

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:

# First, let's load the necessary data
def load_data():
    """Load data from BigQuery"""
    client = bigquery.Client()
    
    # Load listings
    listings_query = """
    SELECT make, model, year, odometer
    FROM `umt-msba.carbitrage.processed_listing_pages`
    WHERE price < 75000
        AND price > 500
        AND odometer < 400000
        AND odometer > 1000
        AND year >= EXTRACT(YEAR FROM CURRENT_DATE()) - 50
        AND year <= EXTRACT(YEAR FROM CURRENT_DATE()) + 1
    """
    
    # Load model coefficients
    models_query = """
    SELECT *
    FROM `umt-msba.carbitrage.lm_lookup_table`
    """
    
    listings = client.query(listings_query).to_dataframe()
    models = client.query(models_query).to_dataframe()
    
    return listings, models


def generate_prediction_grid(listings, models):
    """Generate a grid of predictions for each make/model"""
    # Get distributions for each make/model
    distributions = listings.groupby(['make', 'model']).agg({
    'year': [lambda x: np.percentile(x, 25),
             lambda x: np.percentile(x, 50),
             lambda x: np.percentile(x, 75)],
    'odometer': [lambda x: np.percentile(x, 25),
                 lambda x: np.percentile(x, 50),
                 lambda x: np.percentile(x, 75)]
                 }).reset_index()

    # Rename columns for clarity
    distributions.columns = ['make', 'model', 
                            'year_p25', 'year_p50', 'year_p75',
                            'mile_p25', 'mile_p50', 'mile_p75'] 

    # Cast year columns to integers
    for col in ['year_p25', 'year_p50', 'year_p75']:
        distributions[col] = distributions[col].astype(int)

    # Drop duplicate rows based on 'make', 'model', and year columns
    distributions = distributions.drop_duplicates(subset=['make', 'model', 'year_p25', 'year_p50', 'year_p75'])


    # Create empty list to store all combinations
    rows = []
    
    # For each make/model
    for _, row in distributions.iterrows():
        # Get the model coefficients
        filtered_models = models[
            (models['make'] == row['make']) &
            (models['model'] == row['model'])
            ]
        
        if filtered_models.empty:
            # Skip this make/model if no coefficients are found
            continue
        
        model_coef = filtered_models.iloc[0]
        
        # Generate predictions for each combination
        for year, year_label in zip(
            [row['year_p25'], row['year_p50'], row['year_p75']],
            ['p25', 'p50', 'p75']
        ):
            year = int(year)  # Convert to integer
            for mileage, mile_label in zip(
                [row['mile_p25'], row['mile_p50'], row['mile_p75']],
                ['p25', 'p50', 'p75']
            ):
                predicted_price = (
                    model_coef['intercept'] +
                    model_coef['miles_coeff'] * np.log(mileage + 0.1) +
                    model_coef['condition_coeff'] * 1 +  # Setting condition to 1 (excellent)
                    model_coef['year_coeff'] * year +
                    (model_coef['year_miles_coeff'] * np.log(mileage + 0.1) * year)
                )
                
                rows.append({
                    'make': row['make'],
                    'model': row['model'],
                    'year': year,
                    'mileage': round(mileage, -2),  # Round to nearest thousand
                    'predicted_price': round(predicted_price, 2),
                    'age': pd.Timestamp.now().year - year,
                    'year_percentile': year_label,
                    'mileage_percentile': mile_label,
                    'r_squared': model_coef['r_squared'],
                    'sample_size': model_coef['sample_size']
                })
    
    # Convert to DataFrame
    prediction_df = pd.DataFrame(rows)
    
    # Filter out negative predictions
    prediction_df = prediction_df[prediction_df['predicted_price'] > 0]
    
    return prediction_df


In [ ]:
# Load data
listings, models = load_data()


In [ ]:
listings.head()

In [ ]:

# Generate prediction grid
predictions = generate_prediction_grid(listings, models)


In [ ]:
predictions.query("model == 'legend'")


In [ ]:

# Save to CSV
predictions.to_csv('car_value_predictions.csv', index=False)

# Print some summary statistics
print(f"Generated predictions for {predictions['make'].nunique()} makes and {len(set(zip(predictions['make'], predictions['model'])))} models")
print(f"Total prediction points: {len(predictions)}")



In [ ]:
make = "chevrolet"
model = "corvette"
filtered_data = predictions[(predictions['make'] == make) & (predictions['model'] == model)]

# Create the plot
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=filtered_data, 
    x='mileage', 
    y='predicted_price', 
    hue='year', 
    palette='viridis'
)

# Add labels and title
plt.title(f"Price vs. Mileage for {make.capitalize()} {model.capitalize()} (by Year)")
plt.xlabel("Mileage")
plt.ylabel("Predicted Price")
plt.legend(title='Year')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
make = "honda"
model = "civic"
filtered_data = predictions[(predictions['make'] == make) & (predictions['model'] == model)]

# Create the plot
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=filtered_data, 
    x='mileage', 
    y='predicted_price', 
    hue='year', 
    palette='viridis'
)

# Add labels and title
plt.title(f"Price vs. Mileage for {make.capitalize()} {model.capitalize()} (by Year)")
plt.xlabel("Mileage")
plt.ylabel("Predicted Price")
plt.legend(title='Year')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
predictions.query("make == 'tesla' & model == 'model 3'").sort_values('year')

In [ ]:
# Filter data for two make/models
make_model_combos = [
    ("toyota", "corolla"),
    ("honda", "civic")
]
filtered_data = predictions[
    predictions[['make', 'model']].apply(tuple, axis=1).isin(make_model_combos)
]

# Create the plot
plt.figure(figsize=(12, 8))
sns.lineplot(
    data=filtered_data, 
    x='mileage', 
    y='predicted_price', 
    hue='year', 
    style='model',  # Different styles for models
    palette='viridis',
    markers=True,
    dashes=False
)

# Add labels and title
plt.title("Price vs. Mileage for Selected Make/Models (by Year)")
plt.xlabel("Mileage")
plt.ylabel("Predicted Price")
plt.legend(title='Year and Model', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()
